In [7]:
"""
The wind has changed direction enough to stop sending lava droplets toward you, so you and the elephants exit the cave. As you do, you notice a collection of geodes around the pond. Perhaps you could use the obsidian to create some geode-cracking robots and break them open?

To collect the obsidian from the bottom of the pond, you'll need waterproof obsidian-collecting robots. Fortunately, there is an abundant amount of clay nearby that you can use to make them waterproof.

In order to harvest the clay, you'll need special-purpose clay-collecting robots. To make any type of robot, you'll need ore, which is also plentiful but in the opposite direction from the clay.

Collecting ore requires ore-collecting robots with big drills. Fortunately, you have exactly one ore-collecting robot in your pack that you can use to kickstart the whole operation.

Each robot can collect 1 of its resource type per minute. It also takes one minute for the robot factory (also conveniently from your pack) to construct any type of robot, although it consumes the necessary resources available when construction begins.

The robot factory has many blueprints (your puzzle input) you can choose from, but once you've configured it with a blueprint, you can't change it. You'll need to work out which blueprint is best.

For example:

Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian.
(Blueprints have been line-wrapped here for legibility. The robot factory's actual assortment of blueprints are provided one blueprint per line.)

Using blueprint 1 in the example above, the largest number of geodes you could open in 24 minutes is 9. One way to achieve that is:

== Minute 1 ==
1 ore-collecting robot collects 1 ore; you now have 1 ore.

== Minute 2 ==
1 ore-collecting robot collects 1 ore; you now have 2 ore.

== Minute 3 ==
Spend 2 ore to start building a clay-collecting robot.
1 ore-collecting robot collects 1 ore; you now have 1 ore.
The new clay-collecting robot is ready; you now have 1 of them.

...

== Minute 24 ==
1 ore-collecting robot collects 1 ore; you now have 6 ore.
4 clay-collecting robots collect 4 clay; you now have 41 clay.
2 obsidian-collecting robots collect 2 obsidian; you now have 8 obsidian.
2 geode-cracking robots crack 2 geodes; you now have 9 open geodes.

However, by using blueprint 2 in the example above, you could do even better: the largest number of geodes you could open in 24 minutes is 12.
"""

import sys
import re
import pandas as pd
import numpy as np
import pyomo.environ as pe

def load_blueprints_from_file(filename):
    # Parse lines from file of the following format:
    # Blueprint (BLUEPRINT_ID): Each ore robot costs (ORE_ORE_COST) ore. Each clay robot costs (CLAY_ORE_COST) ore. Each obsidian robot costs (OBSIDIAN_ORE_COST) ore and (OBSIDIAN_CLAY_COST) clay. Each geode robot costs (GEODE_ORE_COST) ore and (GEODE_OBSIDIAN_COST) obsidian.
    # For example:
    # Blueprint 1: Each ore robot costs 1 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
    # Blueprint 2: Each ore robot costs 1 ore. Each clay robot costs 2 ore. Each obsidian robot costs 2 ore and 7 clay. Each geode robot costs 2 ore and 7 obsidian.
    # ...
    # Return a list of blueprints, where each blueprint is a dictionary with the following keys:
    # ore_ore_cost, clay_ore_cost, obsidian_ore_cost, obsidian_clay_cost, geode_ore_cost, geode_obsidian_cost
    blueprints = {}
    with open(filename) as f:
        for line in f:
            if line.startswith("Blueprint"):
                matches = re.search(r"Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.", line)
                if matches:
                    blueprint = {
                      "ore_robot": {
                        "ore": int(matches.group(2)),
                        "clay": 0, "obsidian": 0, "geode": 0
                      },
                      "clay_robot": {
                        "ore": int(matches.group(3)),
                        "clay": 0, "obsidian": 0, "geode": 0
                      },
                      "obsidian_robot": {
                        "ore": int(matches.group(4)),
                        "clay": int(matches.group(5)),
                        "obsidian": 0, "geode": 0
                      },
                      "geode_robot": {
                        "ore": int(matches.group(6)),
                        "clay": 0,
                        "obsidian": int(matches.group(7)),
                        "geode": 0
                      }
                    }
                    blueprints[int(matches.group(1))] = blueprint
                    if len(blueprints) >= 3:
                      break
                else:
                    print("Error: could not parse line: " + line)
    return blueprints

blueprints = load_blueprints_from_file('day19-input.txt')
print("{} blueprints loaded.".format(len(blueprints)))

3 blueprints loaded.


In [8]:
from functools import reduce


total_quality_levels = 0
geodes_collected = []

time = 32

for blueprint_id in blueprints:
    robot_costs = blueprints[blueprint_id]

    # Decision variables
    model = pe.ConcreteModel()

    # Time index for the model
    model.time = pe.Set(initialize=range(0, time + 1))

    # Whether to build a robot of each type at a given time (Constraint: only 1 can be built at a time, ore_build_robot is initialized to 1 at time 0)
    model.ore_build_robot = pe.Var(model.time, domain=pe.Binary, initialize=0)
    model.clay_build_robot = pe.Var(model.time, domain=pe.Binary, initialize=0)
    model.obsidian_build_robot = pe.Var(model.time, domain=pe.Binary, initialize=0)
    model.geode_build_robot = pe.Var(model.time, domain=pe.Binary, initialize=0)

    # At each moment in time, we can calculate the total number of robots of each type by summing the number of robots of that type that have been built up to that time
    model.ore_robots = pe.Expression(model.time, rule=lambda model, t: sum(model.ore_build_robot[i] for i in range(t)))
    model.clay_robots = pe.Expression(model.time, rule=lambda model, t: sum(model.clay_build_robot[i] for i in range(t)))
    model.obsidian_robots = pe.Expression(model.time, rule=lambda model, t: sum(model.obsidian_build_robot[i] for i in range(t)))
    model.geode_robots = pe.Expression(model.time, rule=lambda model, t: sum(model.geode_build_robot[i] for i in range(t)))

    # At each moment in time, we can calculate the number of resources of a type generated by taking the cumulative sum of the number of robots of that type that have been built up to that time
    model.ore_generated = pe.Expression(model.time, rule=lambda model, t: sum(model.ore_robots[i] for i in range(t + 1)))
    model.clay_generated = pe.Expression(model.time, rule=lambda model, t: sum(model.clay_robots[i] for i in range(t + 1)))
    model.obsidian_generated = pe.Expression(model.time, rule=lambda model, t: sum(model.obsidian_robots[i] for i in range(t + 1)))
    model.geode_generated = pe.Expression(model.time, rule=lambda model, t: sum(model.geode_robots[i] for i in range(t + 1)))

    # At each moment in time, we can calculate the number of resources of a type used by multiplying the resource requirements for that resource type of each robot type by the number of robots of that type that have been built up to that time
    model.ore_used = pe.Expression(model.time, rule=lambda model, t: 
        (model.ore_robots[t] - 1 + model.ore_build_robot[t]) * robot_costs['ore_robot']['ore'] + 
        (model.clay_robots[t] + model.clay_build_robot[t]) * robot_costs['clay_robot']['ore'] +
        (model.obsidian_robots[t] + model.obsidian_build_robot[t]) * robot_costs['obsidian_robot']['ore'] +
        (model.geode_robots[t] + model.geode_build_robot[t]) * robot_costs['geode_robot']['ore']
    )
    model.clay_used = pe.Expression(model.time, rule=lambda model, t:
        (model.ore_robots[t] - 1 + model.ore_build_robot[t]) * robot_costs['ore_robot']['clay'] +
        (model.clay_robots[t] + model.clay_build_robot[t]) * robot_costs['clay_robot']['clay'] +
        (model.obsidian_robots[t] + model.obsidian_build_robot[t]) * robot_costs['obsidian_robot']['clay'] +
        (model.geode_robots[t] + model.geode_build_robot[t]) * robot_costs['geode_robot']['clay']
    )
    model.obsidian_used = pe.Expression(model.time, rule=lambda model, t:
        (model.ore_robots[t] - 1 + model.ore_build_robot[t]) * robot_costs['ore_robot']['obsidian'] +
        (model.clay_robots[t] + model.clay_build_robot[t]) * robot_costs['clay_robot']['obsidian'] +
        (model.obsidian_robots[t] + model.obsidian_build_robot[t]) * robot_costs['obsidian_robot']['obsidian'] +
        (model.geode_robots[t] + model.geode_build_robot[t]) * robot_costs['geode_robot']['obsidian']
    )
    model.geode_used = pe.Expression(model.time, rule=lambda model, t:
        (model.ore_robots[t] - 1 + model.ore_build_robot[t]) * robot_costs['ore_robot']['geode'] +
        (model.clay_robots[t] + model.clay_build_robot[t]) * robot_costs['clay_robot']['geode'] +
        (model.obsidian_robots[t] + model.obsidian_build_robot[t]) * robot_costs['obsidian_robot']['geode'] +
        (model.geode_robots[t] + model.geode_build_robot[t]) * robot_costs['geode_robot']['geode']
    )

    # At each moment in time, we can calculate the total number of resources of a type by subtracting the number of resources of that type used from the number of resources of that type generated
    model.ore = pe.Expression(model.time, rule=lambda model, t: model.ore_generated[t] - model.ore_used[t])
    model.clay = pe.Expression(model.time, rule=lambda model, t: model.clay_generated[t] - model.clay_used[t])
    model.obsidian = pe.Expression(model.time, rule=lambda model, t: model.obsidian_generated[t] - model.obsidian_used[t])
    model.geode = pe.Expression(model.time, rule=lambda model, t: model.geode_generated[t] - model.geode_used[t])

    # Constraints
    # Start with 1 ore robot built at time 0
    model.c0 = pe.Constraint(expr=model.ore_build_robot[0] == 1)

    # At any given time, only 1 robot can be built
    model.c2 = pe.ConstraintList()
    for t in model.time:
        model.c2.add(sum([model.ore_build_robot[t], model.clay_build_robot[t], model.obsidian_build_robot[t], model.geode_build_robot[t]]) <= 1)
    # At any given time, the number of resources of a type must be greater than or equal to the cost of the next robot of that type to be built
    for robot_type in robot_costs.keys():
        robot_resource_type = robot_type.split('_')[0]
        for t in model.time:
            if t == 0:
                continue
            mat_constraint_name = robot_type + '_build_robot_mats_' + str(t)
            model.add_component(mat_constraint_name, pe.ConstraintList())
            robot_build_mats = getattr(model, mat_constraint_name)
            for resource_type in robot_costs[robot_type].keys():
                robot_build_mats.add(getattr(model, resource_type)[t - 1] >= robot_costs[robot_type][resource_type] * getattr(model, robot_resource_type + '_build_robot')[t])
        
    # Objective function
    model.obj = pe.Objective(expr=model.geode[time], sense=pe.maximize)

    solver = pe.SolverFactory('glpk')
    solver.solve(model)

    geodes = model.obj()
    geodes_collected.append(geodes)
    quality_level = blueprint_id * geodes

    # Print results
    print("Blueprint {}: {} geodes = quality {}".format(blueprint_id, geodes, quality_level))
    total_quality_levels += quality_level

print("Total quality levels: {}".format(total_quality_levels))
print("Total geodes collected: {} ()".format(reduce(lambda x, y: x * y, geodes_collected), str(geodes_collected)))

Blueprint 1: 31.0 geodes = quality 31.0
Blueprint 2: 8.0 geodes = quality 16.0
Blueprint 3: 17.0 geodes = quality 51.0
Total quality levels: 98.0
Total geodes collected: 4216.0 ()


In [ ]:
results = pd.DataFrame(index=model.time, columns=['ore_build_robot', 'clay_build_robot', 'obsidian_build_robot', 'geode_build_robot', 'ore_robots', 'clay_robots', 'obsidian_robots', 'geode_robots', 'ore', 'clay', 'obsidian', 'geode'])
for t in model.time:
    for col in results.columns:
        try:
            results.loc[t, col] = getattr(model, col)[t]()
        except Exception as e:
            print('Error with column {} at time {}: {}'.format(col, t, e))
display(results)